In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import statsmodels.formula.api as smf
from scipy import stats
from scipy.stats import pearsonr

from helpers.nationality import *

# Initial analysis: How does nationality relate to Oscar nominations?

In [ ]:
movie_df = pd.read_csv('cache/data.csv', sep=',', index_col=0)

We plot the 10 countries with the most Oscar nominations:

In [ ]:
# Getting dataframes of all nominated movies with the "countries"-column exploded (meaning we have one row per actor/movie/country combination)
# exploded_df contains all rows in the dataset exploded by country, nominated_df contains all rows from exploded_df that were nominated for an Oscar
nominated_df, exploded_df = get_nationality_dfs(movie_df)

In [ ]:
sns.set_theme(rc={'figure.figsize':(20, 4)})
plt.xticks(rotation=70)
plt.ylabel('Count')
plt.xlabel('Country')
plt.title('Oscar nominations per country')
sns.countplot(x='country', data=nominated_df, order=nominated_df['country'].value_counts()[:10].index);

We see that the USA is vastly overrepresented. We combine all countries other than the USA and UK into an "others" category

In [ ]:
others_df = nominated_df.copy()
others_df['country'] = nominated_df['country'].apply(lambda country: country if country in ['United States of America', 'United Kingdom'] else 'Others')

In [ ]:
sns.set_theme(rc={'figure.figsize':(6, 4)})
plt.xticks(rotation=70)
plt.ylabel('Count')
plt.xlabel('Country')
plt.title('Oscar nominations per country')
sns.countplot(x='country', data=others_df, order=others_df['country'].value_counts()[:10].index);

In [ ]:
# Get series of countries with their number of nominations adjusted for total number of rows in the dataset. Gives us some idea of the probability of a row from a country being nominated
adjusted_nominations = get_adjusted_nominations(exploded_df, nominated_df)

In [ ]:
sns.set_theme(rc={'figure.figsize':(10, 3)})
plt.xticks(rotation=70)
plt.ylabel('Count')
plt.xlabel('Country')
plt.title('Share of actor/movie pairs from each country being nominated for Oscars')
# Make a barplot for the top 10 countries after adjustment
sns.barplot(data = adjusted_nominations.sort_values(ascending=False)[:10]);

Russia is a surprising winner, could it be an outlier?

In [ ]:
Russia_nominees = exploded_df[(exploded_df['country'] == 'Russia') & (exploded_df['oscar_nominated'] == True)]
print(f"Number of russian nominations: {len(Russia_nominees)}")
Russia_nominees

We see that the Russian nominations are for only one film. This makes Russia a clear outlier.

In general, this analysis is very affected by our data cleaning process. For example, as documented in the data pipeline, we drop most actor/movie rows from India (Which means that we basically disregard Bollywood in this analysis, even though Bollywood would very likely have introduced a significant shift in the Oscard nomination probabilities). Keeping this in mind, we move on to analyze the distribution of American vs. non-American movies in our dataset.

# Binomial analysis USA vs. non-USA

In [ ]:
# movie_character_oscar_rating_df contains a row for each actor/movie pair. We select the non-American actors and compare with the American actors

# All actors/movie rows, American and non-American
total_actors_num = len(movie_df['countries'])
american_total_actors_num = len(movie_df[movie_df['countries'].str.contains('United States of America')])
non_american_total_actors_num = total_actors_num - american_total_actors_num

# All actors/movie rows with an Oscar nomination, American and non-American
total_nominated_actors_num = len(movie_df[movie_df['oscar_nominated'] == True]['countries'])
american_nominations_num = len(movie_df[(movie_df['countries'].str.contains('United States of America')) & (movie_df['oscar_nominated'] == True)])
non_american_nominations_num = total_nominated_actors_num - american_nominations_num

In [ ]:
# Observed probability of American actor getting nominated for a film
p_american = american_nominations_num / american_total_actors_num
p_non_american = non_american_nominations_num / non_american_total_actors_num

# We perform a two-sided hypothesis test for whether non-American actors have the same binomial probability of getting nominated as American ones
stats.binomtest(non_american_nominations_num, non_american_total_actors_num, p_american)

Using alpha=0.05. P-value=0.56 > 0.05. The conclusion from this is that surprisingly, American movies in our dataset do not have a significantly different probability of being nominated for Oscars. Again, this is likely due to our data cleaning, where we remove for example a lot of Indian rows that had NaN-values (And were also likely not Oscar nominated).

In [ ]:
print('Fraction of American actors nominated for an Oscar:',round(p_american, 5))
print('Fraction of non-American actors nominated for an Oscar:', round(p_non_american, 5))

While the probability of winning an Oscar in general is likely higher for American movies, we will keep the non-American movies in this dataset, as the cleaning seems to have removed the expected multicolinearity between country and Oscar nomination.